In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, generate_data_commitment

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')
data_commitment_path = os.path.join('shared/data_commitment.json')

=======================  ZK-STATS FLOW =======================

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)

In [5]:
scales = [6]
selected_columns = ['col_name']
generate_data_commitment(data_path, scales, data_commitment_path)

In [6]:
# Verifier/ data consumer side:
# since have division, scale must be very LARGE, cant use just 0
# So, we allow error rate to be 10% instead of 1%
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    return s.harmonic_mean(x)

error = 0.1
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns,sel_dummy_data_path,verifier_model, verifier_model_path)

/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:172: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


In [7]:
# prover calculates settings, send to verifier
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 3 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.


 <------------- Numerical Fidelity Report (input_scale: 6, param_scale: 6, scale_input_multiplier: 1) ------------->

+-------------+--------------+-----------+-------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error  | median_error | max_error | min_error   | mean_abs_error |

==== Generate & Calibrate Setting ====
scale:  [6]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":6,"param_scale":6,"scale_rebase_multiplier":1,"lookup_range":[-2642,3600],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":14432,"total_assignments":12624,"total_const_size":3312,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,6],"model_input_scales":[6],"module_sizes":{"kzg":[],"poseidon":[14432,[1]]},"required_lookups":[{"GreaterThan":{"a":0.0}},"Abs"],"required_range_checks":[[-32,32],[32,96]],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1709725137745}


In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 1.7187037467956543 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 45.21875
==== Generating Proof ====
proof:  {'instances': [['c371c58876874ace4444c2f2683e0d7bb7ce264f49872aa6deab40635ed90f23', '0100000000000000000000000000000000000000000000000000000000000000', '4e0b000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x07a929a3ffa542c7a53a55b55576a854dc8438504a2a93edea9764b0906cfd9b19127e395a0e2fd1f62924304f49e16c978182bc158c00f3c8545270870f87e40327d3700240fd6bad65ee43a921425bf6810ef643b6c29df3e1da69085c7539002008ff8c8cdb95645e58459cda33cdee83bfe510bbf5b788cfb0ee17c75193001a5d7483a47dd4d48e21de9d0d3c7fa3851cd512243f55a6b77d03a4fcfe8f1bdf72a349549afc4cf86842dab2a813af0b788fc7c87bb3bf580426da50fdd12c42c1bfa4feede0d7bd400ead5e7bc0fca2b08ea144408deaaedf7fd3babd5f03d13ea26e0e7dd6f47cc969549764c18e9ebafc4538cdf48a60e757a9be72eb2bb6ab651efd075ec49a4bb1e7bba845357be6a43e3a7c2123e3d410ab2d8cca20b7e0cf070

In [9]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, data_commitment_path)
print("Verifier gets result:", res)

Verifier gets result: [45.21875]
